# 08 Driver Characteristics Extraction

Extract driver performance vs teammates from 2024-2025 data.

**Features:**
- Session-first processing (load once, extract all drivers)
- Dynamic driver detection (no hardcoded lineups)
- Separate quali and race extraction
- Checkpoint system for resumability
- Clean error handling

**Outputs:**
- `driver_quali_characteristics.json` - Qualifying pace vs teammate
- `driver_race_characteristics.json` - Race pace vs teammate
- `extraction_progress.json` - Checkpoint for incremental updates

## Setup

In [1]:
import fastf1 as ff1
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from collections import defaultdict
from itertools import combinations
import time

# Suppress all API-related noise
# Cache is working - these just hide misleading 429 errors
import logging
logging.getLogger("fastf1").setLevel(logging.CRITICAL)
logging.getLogger("requests_cache").setLevel(logging.CRITICAL)
logging.getLogger("urllib3").setLevel(logging.CRITICAL)
logging.getLogger("requests").setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

# Enable cache (data loads from cache when available)
ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

## Config

In [2]:
# Extract from last 2 years
SEASONS = [2024, 2025]

# Output paths
OUTPUT_DIR = Path('../data/processed/testing_files/driver_characteristics')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

QUALI_OUTPUT = OUTPUT_DIR / 'driver_quali_characteristics.json'
RACE_OUTPUT = OUTPUT_DIR / 'driver_race_characteristics.json'
PROGRESS_FILE = OUTPUT_DIR / 'extraction_progress.json'

print(f"Extracting from seasons: {SEASONS}")
print(f"Output directory: {OUTPUT_DIR}")

Extracting from seasons: [2024, 2025]
Output directory: ../data/processed/testing_files/driver_characteristics


## Helper Functions

In [3]:
def load_progress():
    """Load extraction progress from checkpoint file."""
    if not PROGRESS_FILE.exists():
        return set()
    
    with open(PROGRESS_FILE, 'r') as f:
        data = json.load(f)
    
    return set(data.get('completed_sessions', []))


def save_progress(completed_sessions, quali_data, race_data):
    """Save current progress and intermediate results."""
    progress = {
        'last_updated': datetime.now().isoformat(),
        'completed_sessions': list(completed_sessions),
        'total_sessions': len(completed_sessions)
    }
    
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f, indent=2)
    
    # Save intermediate results
    with open(QUALI_OUTPUT, 'w') as f:
        json.dump(quali_data, f, indent=2, default=str)
    
    with open(RACE_OUTPUT, 'w') as f:
        json.dump(race_data, f, indent=2, default=str)


def safe_load_session(year, event_name, session_type):
    """
    Load session from cache or API.
    
    Note: 429 errors are suppressed - cache is working properly.
    """
    try:
        session = ff1.get_session(year, event_name, session_type)
        
        # Load only what I need
        if session_type == 'Q':
            session.load(laps=False, telemetry=False, weather=False)
        else:  # Race
            session.load(laps=True, telemetry=False, weather=False)
        
        return session
        
    except Exception as e:
        # Only show real errors (not 429s which are handled by cache)
        if '429' not in str(e):
            print(f"  🔴 Failed to load: {e}")
        return None


def get_session_id(year, event_name, session_type):
    """Create unique session identifier."""
    return f"{year}_{event_name}_{session_type}"

## Extract Qualifying Data

In [4]:
def extract_quali_session(session, year, event_name):
    """
    Extract all valid driver-teammate comparisons from qualifying.
    Handles teams with any number of drivers.
    """
    if not hasattr(session, 'results') or session.results is None:
        return []
    
    results = session.results
    comparisons = []
    
    # Group drivers by team
    teams = results.groupby('TeamName')
    
    for team_name, team_results in teams:
        
        # Get all drivers with their best qualifying times
        drivers_with_times = []
        
        for _, driver_row in team_results.iterrows():
            driver_abbr = driver_row['Abbreviation']
            
            # Get best time from Q3 > Q2 > Q1
            best_time = None
            for col in ['Q3', 'Q2', 'Q1']:
                if col in driver_row.index and pd.notna(driver_row[col]):
                    best_time = driver_row[col]
                    break
            
            # Only include drivers with valid times
            if best_time is not None:
                drivers_with_times.append({
                    'abbr': driver_abbr,
                    'time': best_time.total_seconds()
                })
        
        # Need at least 2 drivers with valid times to compare
        if len(drivers_with_times) < 2:
            continue
        
        # Extract all pairwise comparisons
        for d1, d2 in combinations(drivers_with_times, 2):
            
            # Calculate ratios
            d1_ratio = d1['time'] / d2['time']
            d2_ratio = d2['time'] / d1['time']
            
            # Add both directions
            comparisons.append({
                'driver': d1['abbr'],
                'teammate': d2['abbr'],
                'team': team_name,
                'year': year,
                'event': event_name,
                'ratio': d1_ratio,
                'driver_time': d1['time'],
                'teammate_time': d2['time']
            })
            
            comparisons.append({
                'driver': d2['abbr'],
                'teammate': d1['abbr'],
                'team': team_name,
                'year': year,
                'event': event_name,
                'ratio': d2_ratio,
                'driver_time': d2['time'],
                'teammate_time': d1['time']
            })
    
    return comparisons

## Extract Race Data

In [5]:
def extract_race_session(session, year, event_name):
    """
    Extract all valid driver-teammate comparisons from race.
    Uses median lap time to compare teammates.
    """
    if not hasattr(session, 'laps') or session.laps is None:
        return []
    
    laps = session.laps
    comparisons = []
    
    # Group drivers by team
    teams = laps.groupby('Team')
    
    for team_name, team_laps in teams:
        
        # Get median lap time for each driver
        drivers_with_times = []
        
        for driver_abbr in team_laps['Driver'].unique():
            driver_laps = team_laps[team_laps['Driver'] == driver_abbr]
            
            # Filter valid laps
            valid_laps = driver_laps[
                (driver_laps['LapTime'].notna()) &
                (driver_laps['Stint'] > 0)
            ]
            
            if len(valid_laps) < 3:
                continue
            
            # Calculate median lap time
            lap_times = valid_laps['LapTime'].apply(lambda x: x.total_seconds())
            median_time = lap_times.median()
            
            drivers_with_times.append({
                'abbr': driver_abbr,
                'median_time': median_time
            })
        
        # Need at least 2 drivers with valid times
        if len(drivers_with_times) < 2:
            continue
        
        # Extract all pairwise comparisons
        for d1, d2 in combinations(drivers_with_times, 2):
            
            # Calculate ratios
            d1_ratio = d1['median_time'] / d2['median_time']
            d2_ratio = d2['median_time'] / d1['median_time']
            
            # Add both directions
            comparisons.append({
                'driver': d1['abbr'],
                'teammate': d2['abbr'],
                'team': team_name,
                'year': year,
                'event': event_name,
                'ratio': d1_ratio,
                'driver_median': d1['median_time'],
                'teammate_median': d2['median_time']
            })
            
            comparisons.append({
                'driver': d2['abbr'],
                'teammate': d1['abbr'],
                'team': team_name,
                'year': year,
                'event': event_name,
                'ratio': d2_ratio,
                'driver_median': d2['median_time'],
                'teammate_median': d1['median_time']
            })
    
    return comparisons

## Run Extraction

In [6]:
# Run this cell before running extraction
import os

# Remove old format files
for file in [QUALI_OUTPUT, RACE_OUTPUT, PROGRESS_FILE]:
    if file.exists():
        os.remove(file)
        print(f"Removed old file: {file}")

print("Starting fresh extraction...")

Starting fresh extraction...


In [7]:
# Load checkpoint
completed_sessions = load_progress()

# Initialize data storage
quali_comparisons = []
race_comparisons = []

# Load existing data if any
if QUALI_OUTPUT.exists():
    with open(QUALI_OUTPUT, 'r') as f:
        quali_comparisons = json.load(f)

if RACE_OUTPUT.exists():
    with open(RACE_OUTPUT, 'r') as f:
        race_comparisons = json.load(f)

print(f"Resuming from checkpoint: {len(completed_sessions)} sessions already processed\n")

# Get schedule for each season
total_sessions = 0
sessions_to_process = []

for year in SEASONS:
    schedule = ff1.get_event_schedule(year)
    
    for _, event in schedule.iterrows():
        event_name = event['EventName']
        event_format = event.get('EventFormat', 'conventional')
        is_sprint_weekend = 'sprint' in str(event_format).lower()
        
        # Always process Q and R
        for session_type in ['Q', 'R']:
            session_id = get_session_id(year, event_name, session_type)
            if session_id not in completed_sessions:
                sessions_to_process.append((year, event_name, session_type, session_id))
                total_sessions += 1
        
        # Only add sprint sessions if sprint weekend
        if is_sprint_weekend:
            for session_type in ['SQ', 'S']:
                session_id = get_session_id(year, event_name, session_type)
                if session_id not in completed_sessions:
                    sessions_to_process.append((year, event_name, session_type, session_id))
                    total_sessions += 1

print(f"Total sessions to process: {total_sessions}\n")

# Process each season
for year in SEASONS:
    print(f"{'='*70}")
    print(f"Processing {year}")
    print(f"{'='*70}\n")
    
    schedule = ff1.get_event_schedule(year)
    
    for _, event in schedule.iterrows():
        event_name = event['EventName']
        
        # Check if this is a sprint weekend
        event_format = event.get('EventFormat', 'conventional')
        is_sprint_weekend = 'sprint' in str(event_format).lower()
        
        # Process qualifying
        quali_id = get_session_id(year, event_name, 'Q')
        if quali_id not in completed_sessions:
            print(f"[{year}] {event_name} - Qualifying")
            session = safe_load_session(year, event_name, 'Q')
            if session:
                comps = extract_quali_session(session, year, event_name)
                quali_comparisons.extend(comps)
                completed_sessions.add(quali_id)
                print(f"  🟢 Extracted {len(comps) // 2} driver pairs")
        
        # Process sprint sessions ONLY if sprint weekend
        if is_sprint_weekend:
            # Sprint qualifying/shootout
            sq_id = get_session_id(year, event_name, 'SQ')
            if sq_id not in completed_sessions:
                print(f"[{year}] {event_name} - Sprint Qualifying")
                session = safe_load_session(year, event_name, 'SQ')
                if session:
                    comps = extract_quali_session(session, year, event_name)
                    quali_comparisons.extend(comps)
                    completed_sessions.add(sq_id)
                    print(f"  🟢 Extracted {len(comps) // 2} driver pairs")
            
            # Sprint race
            sprint_id = get_session_id(year, event_name, 'S')
            if sprint_id not in completed_sessions:
                print(f"[{year}] {event_name} - Sprint")
                session = safe_load_session(year, event_name, 'S')
                if session:
                    comps = extract_race_session(session, year, event_name)
                    race_comparisons.extend(comps)
                    completed_sessions.add(sprint_id)
                    print(f"  🟢 Extracted {len(comps) // 2} driver pairs")
        
        # Process race
        race_id = get_session_id(year, event_name, 'R')
        if race_id not in completed_sessions:
            print(f"[{year}] {event_name} - Race")
            session = safe_load_session(year, event_name, 'R')
            if session:
                comps = extract_race_session(session, year, event_name)
                race_comparisons.extend(comps)
                completed_sessions.add(race_id)
                print(f"  🟢 Extracted {len(comps) // 2} driver pairs")
        
        # Save checkpoint every 10 sessions
        if len(completed_sessions) % 10 == 0:
            save_progress(completed_sessions, quali_comparisons, race_comparisons)
            print(f"\n  💾 Checkpoint: {len(completed_sessions)} sessions processed\n")

# Final save
save_progress(completed_sessions, quali_comparisons, race_comparisons)

print(f"\n{'='*70}")
print("Extraction complete")
print(f"{'='*70}")
print(f"Quali comparisons: {len(quali_comparisons)}")
print(f"Race comparisons: {len(race_comparisons)}")

Resuming from checkpoint: 0 sessions already processed

Total sessions to process: 124

Processing 2024

[2024] Pre-Season Testing - Qualifying
  🟢 Extracted 10 driver pairs
[2024] Pre-Season Testing - Race
  🟢 Extracted 10 driver pairs
[2024] Bahrain Grand Prix - Qualifying
  🟢 Extracted 10 driver pairs
[2024] Bahrain Grand Prix - Race
  🟢 Extracted 10 driver pairs
[2024] Saudi Arabian Grand Prix - Qualifying
  🟢 Extracted 9 driver pairs
[2024] Saudi Arabian Grand Prix - Race
  🟢 Extracted 9 driver pairs
[2024] Australian Grand Prix - Qualifying
  🟢 Extracted 9 driver pairs
[2024] Australian Grand Prix - Race
  🟢 Extracted 9 driver pairs
[2024] Japanese Grand Prix - Qualifying
  🟢 Extracted 10 driver pairs
[2024] Japanese Grand Prix - Race
  🟢 Extracted 8 driver pairs

  💾 Checkpoint: 10 sessions processed

[2024] Chinese Grand Prix - Qualifying
  🟢 Extracted 10 driver pairs
[2024] Chinese Grand Prix - Sprint Qualifying
  🟢 Extracted 10 driver pairs
[2024] Chinese Grand Prix - Sprint


## Summary Statistics

In [8]:
# Convert to DataFrames for analysis
quali_df = pd.DataFrame(quali_comparisons)
race_df = pd.DataFrame(race_comparisons)

print("\nQUALIFYING SUMMARY")
print("="*50)
print(f"Total comparisons: {len(quali_df)}")
print(f"Unique drivers: {quali_df['driver'].nunique()}")
print(f"Sessions covered: {quali_df.groupby(['year', 'event']).ngroups}")

print("\nDrivers with most comparisons:")
print(quali_df['driver'].value_counts().head(10))

print("\n\nRACE SUMMARY")
print("="*50)
print(f"Total comparisons: {len(race_df)}")
print(f"Unique drivers: {race_df['driver'].nunique()}")
print(f"Sessions covered: {race_df.groupby(['year', 'event']).ngroups}")

print("\nDrivers with most comparisons:")
print(race_df['driver'].value_counts().head(10))


QUALIFYING SUMMARY
Total comparisons: 1214
Unique drivers: 27
Sessions covered: 50

Drivers with most comparisons:
driver
RUS    62
PIA    62
LEC    62
GAS    62
HAM    62
NOR    62
ALO    61
STR    61
VER    61
TSU    61
Name: count, dtype: int64


RACE SUMMARY
Total comparisons: 1164
Unique drivers: 27
Sessions covered: 50

Drivers with most comparisons:
driver
LEC    61
HAM    60
RUS    60
NOR    59
SAI    59
PIA    59
VER    58
HUL    58
TSU    57
OCO    57
Name: count, dtype: int64
